# Prediction Challenge

## Module importieren

In [1]:
import pandas as pd
import json
import requests

## Helfer-Funktionen

In [2]:
def compete(name, predictions):
    predictions = list(predictions.values)
    user = name

    params = {
        'user': user,
        'predictions': json.dumps(predictions)
    }

    url = 'https://efichallenge2021.azurewebsites.net/compete'
    response = requests.post(url, data=params)
    
    try:
        answer = response.json()
        print('Server-Antwort: {}'.format(answer['message']))
        print('Punktzahl: {}'.format(answer['score']))
    except:
        print(response.content)

## Datensätze laden

In [3]:
df_training = pd.read_csv('data/steam_training.csv')
df_test = pd.read_csv('data/steam_test.csv')

## Datensatzbeschreibung

Der Datensatz enthält auf Steam erhältliche Spiele (1997 - 2019)

- **appid** : the ID number of the app
- **name** : the title of the game
- **release_date** : the date of the game’s release
- **english** : If the game is available in english or not (1 = yes, 0 = no)
- **developer** : the name of the game’s developer
- **publisher** : the name of the game’s publisher
- **platforms** : the operating system which can play this game (windows, mac. or linux)
- **required_age** : the minimum age required before you can play this game
- **categories** : whether the game is a single-player, multi-player, or online
- **genres** : the genre of the game, if it is not a game then what kind of software it is
- **steamspy_tags** : the genre of the games as categorized by steamspy
- **achievements** : the number of achievements of a game
- **positive_ratings** : number of positive ratings of a game
- **negative_ratings** : number of negative ratings of a game
- **average_playtime** : the average playtime in hours
- **median_playtime** : the median playtime in hours
- **owners** : the number of people who owns the game; the type is categorical rather than numerical
- **price** : the price of the game in poundsterling, this will be converted to USD

## Challenge

- df_training enthält 5935 Spiele. Mit ihnen sollst du ein Modell trainieren, welches die Anzahl Besitzer (*owners*) vorhersagt.
- df_test enthält 2544 Spiele. Der Datensatz hat dieselbe Struktur wie *df_training*, allerdings fehlt der Eintrag der Besitzer (*owners*)
- Die Challenge ist es, die *owners* in *df_test* mit dem trainierten Modell vorherzusagen. Dies kannst du mit dem Aufruf unter **Test Prediction** überprüfen. Die Person mit der höchsten Score (min = 0, max = 100) gewinnt.

## Hilfestellungen 

### Modelle importieren

- Wir haben bisher die Methoden *Decision Tree*, *KNN* und *Ensemble Learning* kennen gelernt. Es ist dir überlassen, ob du nur eine der Methoden oder einen Mix verwenden möchtest. Die nötigen Module für die Methoden musst du jedoch importieren, bevor du sie verwenden kannst. Hilfe dazu findest du in den Beispiel-Notebooks **07 DecisionTrees_solutions.ipynb**, **neighbour_hool.ipynb** und **ensemble_hool.ipynb**.
- Wenn du beispielsweise KNN verwenden möchtest, musst du das nötige Modul importieren:
> from sklearn.neighbors import KNeighborsClassifier
- Wenn du für die Hyperparametersuche den Trainingsset Samplen möchtest, musst du wiederum das nötige Modul importieren:
> from sklearn.model_selection import train_test_split
- usw.

### Passende spalten erstellen
- Bestimmte Spalten enthalten bereits nummerische Werte, andere enthalten jedoch Strings (Text).
- Beispielsweise enthält die Spalte *platforms* die erlaubten Betriebssysteme als String. Wenn du die Platform für dein Modell verwenden möchtest, kannst du mit den Befehlen in der folgenden Zelle die nötigen Spalten erstellen. Die Befehle erstellen jeweils eine neue Spalte, welche den Wert 1 enthält, falls das Schlüsselwort in der Spalte *platforms* vorkommt und 0, wenn nicht. **Denke daran: du musst das für die Trainingsdaten, aber auch für die Testdaten machen. Sie müssen die gleiche Form behalten**
- Dieses Vorgehen ist auch auf andere Spalten wie z.B. *categories* oder *publisher* anwendbar.

In [12]:
df_training['has_windows'] = df_training['platforms'].str.contains('windows').astype(int)
df_training['has_mac'] = df_training['platforms'].str.contains('mac').astype(int)
df_training['has_linux'] = df_training['platforms'].str.contains('linux').astype(int)

df_training

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,...,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,has_windows,has_mac,has_linux
0,376750,World War II GI,2015-11-13,1,TNT Team,Retroism;Nightdive Studios,windows;linux,0,Single-player,Action,...,0,20,17,0,0,20000-50000,4.99,1,0,1
1,476650,The Silver Case,2016-10-06,1,GRASSHOPPER MANUFACTURE INC.;AGM PLAYISM,GRASSHOPPER MANUFACTURE INC.,windows;mac;linux,0,Single-player;Steam Achievements;Steam Trading...,Adventure,...,25,176,26,0,0,20000-50000,14.99,1,1,1
2,453100,Frederic: Resurrection of Music Director's Cut,2016-04-21,1,Forever Entertainment S. A.,Forever Entertainment S. A.,windows;mac;linux,0,Single-player;Multi-player;Cross-Platform Mult...,Action;Casual;Indie,...,25,197,41,290,323,50000-100000,4.79,1,1,1
3,244710,Shelter,2013-08-28,1,Might and Delight,Might and Delight,windows;mac,0,Single-player;Steam Trading Cards;Partial Cont...,Adventure;Indie,...,0,2774,479,77,87,200000-500000,6.99,1,1,0
4,607890,Sword Art Online: Hollow Realization Deluxe Ed...,2017-10-27,1,AQURIA,BANDAI NAMCO Entertainment,windows,0,Single-player;Online Multi-Player;Online Co-op...,Action;Adventure;Casual;RPG,...,53,2860,707,605,941,100000-200000,39.99,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5930,464540,Colosse,2016-04-21,1,Colosse Team,Fire Panda Ltd.,windows,0,Single-player;Full controller support,Adventure,...,0,106,24,0,0,50000-100000,0.00,1,0,0
5931,423880,Carpe Diem,2015-11-30,1,Moonlit Works,Moonlit Works,windows;mac;linux,0,Single-player;Steam Achievements;Steam Cloud,Casual;Free to Play;Indie,...,1,6791,793,10,11,200000-500000,0.00,1,1,1
5932,273600,Descent: FreeSpace – The Great War,2014-05-07,1,Volition Inc.,Interplay Inc.,windows,0,Single-player;Multi-player;Includes level editor,Action,...,0,123,30,0,0,20000-50000,6.99,1,0,0
5933,219190,Ultratron,2013-03-18,1,Puppygames,Puppygames,windows;mac;linux,0,Single-player;Shared/Split Screen;Steam Achiev...,Action;Indie,...,67,545,60,84,84,50000-100000,7.19,1,1,1


## Modell Trainieren

- Trainiere ein Modell mit Hilfe von **df_training**

In [5]:
#TODO

## Modell anwenden
- Sage die "owners" für **df_test** voraus

In [9]:
#TODO
df_test['predictions'] = 'test'

## Modell testen

In [11]:
name = 'x'
predictions = df_test['predictions']

compete(name, predictions)

Server-Antwort: Predictions received
Punktzahl: 0.0


**Resultate:** https://efichallenge2021.azurewebsites.net/results